<a href="https://colab.research.google.com/github/mvarentsov/UClim4HSE-2025/blob/main/scripts/Lect07_run_TEB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pandas as pd

%pip install f90nml
import f90nml

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Скачивание модели из репозитория
Остророжно, директорая удаляется и создается заново!

In [ ]:
%cd /content/drive/MyDrive/
%pwd

os.makedirs ('UClim4HSE', exist_ok = True)

%cd /content/drive/MyDrive/UClim4HSE/
%pwd

if os.path.isdir ('TEB_open_source_v3_sfx8.1_orig_namelist'):
   %rm -r 'TEB_open_source_v3_sfx8.1_orig_namelist'

print ('cloning repository')
!git clone https://github.com/mkolennikova/TEB_open_source_v3_sfx8.1_orig_namelist.git


### Компиляция модели

In [ ]:
%cd '/content/drive/MyDrive/UClim4HSE/TEB_open_source_v3_sfx8.1_orig_namelist'
!make clean
!make all

#### Проверка наличия исполняемого файла

In [ ]:
%ll *.exe

#### Запуск исполняемого файла модели

In [ ]:
!./tester.exe

#### Проверим наличие результатов моделирования



In [ ]:
%ll output_Moscow/

#### Чтение и визуализация результатов моделирования

In [ ]:
def read_TEB_output (output_dir, namelist_path):
  file_paths = [f for f in os.listdir(output_dir)]

  output_df = pd.DataFrame()
  for out_file in file_paths:
    param = pd.read_csv(output_dir+'/'+out_file, header=None)
    output_df[out_file[:-4]] = param

  #display(output_df)

  namelist = f90nml.read(namelist_path) #'src_driver/namelist_forcing.nml')

  #display (namelist['tebforcing'])

  t1 = pd.Timestamp (namelist['tebforcing']['iyear'], namelist['tebforcing']['imonth'],  namelist['tebforcing']['iday'])
  t2 = t1 + pd.Timedelta (seconds=namelist['tebforcing']['timestep']) * (namelist['tebforcing']['num_timesteps']-2)

  output_df.index = pd.date_range(t1, t2, freq=pd.Timedelta (seconds=namelist['tebforcing']['timestep']))
  return output_df

teb_output = read_TEB_output ('output_Moscow', 'src_driver/namelist_forcing.nml')
display(c)




In [ ]:
pd.options.plotting.backend = "plotly"

fig = teb_output[['T_CANYON', 'TI_BLD']].plot()
fig.show()